In [56]:
import json
from nltk import word_tokenize, sent_tokenize
import numpy as np
nk.download('punkt')

[nltk_data] Downloading package punkt to /home/stefan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [3]:
DATA_PATH = '../data/GermanFakeNC.json'

In [9]:
def read_data(path):
    with open(path) as json_file:
        return json.load(json_file)

In [80]:
def count_matches(false_statement, sentence, count):
    count = 0
    sent_copy = sentence[:]
    for w in false_statement:
        if w in sent_copy:
            count += 1
            sent_copy.remove(w)
    return count

labeled_sentences = []
for article in read_data(DATA_PATH):
    # Concatenate article text
    text = article['Title'] + article['Teaser'] + article['Text']
    
    # Seperate article text into sentences
    sentences = sent_tokenize(text, language='german')
    
    # Seperate sentences into words
    sentences = [word_tokenize(sent, language='german') for sent in sentences]
    
    # Label sentences
    # The sentences matching the most tokens with a false statement will be labeled as False
    false_statements = [article['False_Statement_1'], article['False_Statement_2'], article['False_Statement_3']]     
    labeled_article_sentences = [[s, True] for s in sentences]
    for fs in false_statements:
        if fs != '':
            fs_words = word_tokenize(fs, language='german')
            matches = [count_matches(fs_words, s, 0) for s in sentences]
            m = max(matches)
            max_indexes = [i for i, j in enumerate(matches) if j == m]
            for mi in max_indexes:
                labeled_article_sentences[mi][1] = False
            
    labeled_sentences = labeled_sentences + labeled_article_sentences
print(labeled_sentences[4])

[['–', 'Mai', '2013', ':', 'Auf', 'der', 'griechischen', 'Ferieninsel', 'Korfu', 'überfällt', 'Hussein', 'Khavari', 'eine', 'Studentin', 'und', 'wirft', 'sie', 'eine', 'hohe', 'Klippe', 'hinab', '.'], True]


### Options to match fake statements to sentences
* Test if sentence is in fake statement: matched 53.7% of false statements 
* Seperate into word tokens and test if some percetage of words is in a false statement
* Label sentence with most matching words as false statement

In [79]:
tf_stats = 0
for a in read_data(DATA_PATH):
    for number in ['1','2','3']:
        if a['False_Statement_' + number] != '':
            tf_stats += 1
            
cf_stats = len(list(filter(lambda x: not x[1], labeled_sentences))) 
print("Number of all statements {}".format(len(labeled_sentences)))
print("True number of false statements {}".format(tf_stats))
print("Classified number of false statements {} ({:.1f}%)".format(cf_stats, (cf_stats * 100) / tf_stats))

Number of all statements 11252
True number of false statements 974
Classified number of false statements 951 (97.6%)
